<a href="https://colab.research.google.com/github/wolfinwallst/Machine_Learning_Deep_basis/blob/main/NLP_RNN_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문자 레벨 RNN (character-level RNN): RNN의 입출력 단위가 단어 레벨 (word-level)이 아닌 문자 레벨인 경우

`Char RNN`

아래에선, 문자 단위 RNN을 다대다 구조로 구현한다:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN을 구현해보자:

In [2]:
input_str = 'apple'
label_str = 'pple!'

# 입력 데이터와 레이블 데이터에서 중복을 제거한 문자들의 집합인 문자 집합(vocabulary)을 생성한다
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)

print('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [3]:
print(char_vocab)

['!', 'a', 'e', 'l', 'p']


In [4]:
# hyperparameters

# 입력 크기는 문자 집합의 크기 (입력은 1-hot 벡터를 사용할 예정이므로 이렇게 설정)
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [5]:
# 문자 집합에 고유한 정수를 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [6]:
char_to_index.items()

dict_items([('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)])

In [7]:
# 나중에 예측 결과를 다시 문자 시퀀스로 보기위해서
# 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만든다.
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [8]:
# 입력/레이블 데이터의 각 문자들을 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]

print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [9]:
"""
PyTorch의 nn.RNN()은 기본적으로 3차원 텐서를 입력받으므로 배치 차원을 추가해주자
"""
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있다.
x_data = [x_data]
y_data = [y_data]

print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [10]:
# [one-hot encoding 구현 트릭] 이니 기억해두자!

# vocab_size = 5 로 위에서 설정했다.
"""
입력 시퀀스의 각 문자들을 원-핫 벡터로 변환
x_data에 있는 각 값 x를 one-hot encoding 형태로 변환

vocab_size x vocab_size 크기의 단위 행렬의 x 번째 행을 선택
"""
x_one_hot = [np.eye(vocab_size)[x] for x in x_data] # np.eye(n) 으로 n x n 단위행렬 생성
print(x_one_hot)

# x_data 와 비교하면 같은 위치에만 1이고 따라서 x_one_hot이 x_data의 원-핫 인코딩임을 알 수 있다.

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [12]:
# 입력 데이터와 레이블 데이터를 텐서로 바꾼다.
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블 데이터의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블 데이터의 크기 : torch.Size([1, 5])


<ipython-input-12-4f1afd07e5bb>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X = torch.FloatTensor(x_one_hot)


이제 RNN 모델을 구현한다:

In [13]:
# RNN 모델을 구현하는 클래스
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [14]:
net = Net(input_size, hidden_size, output_size)
print(net)

outputs = net(X)
print(outputs.shape) # 3D 텐서 (배치 차원, 시점(timesteps), 출력 크기)

Net(
  (rnn): RNN(5, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)
torch.Size([1, 5, 5])


In [15]:
"""
나중에 정확도를 측정할 때, 이를 모두 펼쳐서 계산하게 되는데,
이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만든다.
"""
print(outputs.view(-1, input_size).shape) # 2D 텐서로 변환

torch.Size([5, 5])


In [16]:
"""
마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정
"""
# 레이블 데이터의 크기
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [18]:
num_epochs = 100

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)]) # squeeze()로 result중 크기가 1인 차원(axis)을 자동으로 제거
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)


0 loss:  1.6339318752288818 prediction:  [[1 4 4 1 1]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  appaa
1 loss:  1.3643710613250732 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppep
2 loss:  1.1675583124160767 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppep
3 loss:  0.9615438580513 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppep
4 loss:  0.7863997220993042 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.6365571618080139 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.4991140365600586 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
7 loss:  0.3859456479549408 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
8 loss:  0.27646633982658386 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.20868992805480957 prediction:  [[4 4 3 2 0]] t

# 2. 더 많은 데이터로 학습한 문자 단위 RNN (`Char RNN`)

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence)) # 중복을 제거한 문자 집합
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 고유한 정수 인코딩

print(char_dic) # 공백도 하나의 원소가 된다

{'n': 0, 'r': 1, 'c': 2, 'w': 3, ' ': 4, ',': 5, 'm': 6, 'l': 7, 'a': 8, 'o': 9, '.': 10, 'e': 11, 's': 12, "'": 13, 'd': 14, 'i': 15, 'p': 16, 'y': 17, 'b': 18, 'u': 19, 'g': 20, 'k': 21, 't': 22, 'h': 23, 'f': 24}


In [21]:
print(char_set)

['n', 'r', 'c', 'w', ' ', ',', 'm', 'l', 'a', 'o', '.', 'e', 's', "'", 'd', 'i', 'p', 'y', 'b', 'u', 'g', 'k', 't', 'h', 'f']


In [22]:
dic_size = len(char_dic) # 매 시점마다 들어갈 입력의 크기_입력은 원-핫 벡터로 사용
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [23]:
# 하이퍼파라미터 설정
hidden_size = dic_size # hidden state 크기를 input 크기로 줬는데, 다른 값을 선택해도 무방
sequence_length = 10 # 임의로 설정 / 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정
learning_rate = 0.1

x_data = []
y_data = []

"""
샘플인 long sentence를 sequence_length(=10) 단위만큼 잘라서
윈도우(of size 1)를 이동해 x_data, y_data 생성
"""
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str]) # x str to idx
    y_data.append([char_dic[c] for c in y_str]) # y str to idx

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [24]:
# 170개의 샘플이 생성됨
print('x_data 개수: ', len(x_data), '& y_data 개수: ', len(y_data))

x_data 개수:  170 & y_data 개수:  170


In [25]:
# list형이라 np.shape(리스트 명) 으로 shape을 보자
print(np.shape(x_data), ' & ', np.shape(y_data))

(170, 10)  &  (170, 10)


In [26]:
print(x_data[0]) # 첫번째 샘플의 입력 데이터 / if you wan에 해당
print(y_data[0]) # 첫번째 샘플의 레이블 데이터 /  f you want에 해당

[15, 24, 4, 17, 9, 19, 4, 3, 8, 0]
[24, 4, 17, 9, 19, 4, 3, 8, 0, 22]


In [27]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # 입력 시퀀스에 원-핫 인코딩 수행 / x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot) # 텐서로 변환
Y = torch.LongTensor(y_data)

# 10은 sequence_length, 25는 char_dic 크기, 즉 전체 sentence에 사용된 문자의 개수
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([170, 10, 25])
레이블의 크기: torch.Size([170, 10])


In [28]:
x_one_hot[0]
print(X[0].shape, Y[0].shape) # 0 if you wan -> f you want

torch.Size([10, 25]) torch.Size([10])


In [29]:
len(x_one_hot)

170

In [30]:
print(x_one_hot[0])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


In [31]:
# 첫번째 샘플의 원-핫 인코딩 결과
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,

tensor([
[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # i

[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # f

[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # 공백

[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # y

[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], # o

[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # y

[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], # 공백

[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], # w

[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], # a

[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]) # n



In [32]:
# 레이블 데이터의 첫번째 샘플의 원-핫 인코딩 결과
# f you want에 해당
print(Y[0])

tensor([24,  4, 17,  9, 19,  4,  3,  8,  0, 22])


# 2) 모델 구현하기

In [33]:
"""
vannila RNN w/ 2 layers
"""
class Net(torch.nn.Module):
    # 현재 hidden_size는 dic_size와 동일
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size, 2) # 층 2개 사용

In [34]:
net

Net(
  (rnn): RNN(25, 25, num_layers=2, batch_first=True)
  (fc): Linear(in_features=25, out_features=25, bias=True)
)

In [35]:
"""
비용 함수와 옵티마이저
"""
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape) # 3D 텐서, [170, 10, 25] 크기로, 배치 차원 x 시점(timesteps) x 출력 크기

torch.Size([170, 10, 25])


In [36]:
"""
나중에 정확도를 측청할 때, 이를 모두 펼쳐서 계산하는데
이때 view를 사용하여 배치 차원과 시점 차원을 하나로 만든다.
"""
print(outputs.view(-1, dic_size).shape) # 2D 텐서로 변환

torch.Size([1700, 25])


In [37]:
print(Y.shape) # 레이블 데이터의 크기
print(Y.view(-1).shape)
print(Y.view(-1))

torch.Size([170, 10])
torch.Size([1700])
tensor([24,  4, 17,  ..., 11,  8, 10])


In [38]:
for i in range(num_epochs):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2) # results의 텐서 크기는 (170, 10)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오고
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음부턴 마지막 글자만 추가
            predict_str += char_set[result[-1]]

print(predict_str)

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [39]:
results.shape

torch.Size([170, 10])